In [1]:
!nvidia-smi

Tue May 25 19:09:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## get transformers library

In [2]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 73444, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 73444 (delta 111), reused 155 (delta 85), pack-reused 73239
Receiving objects: 100% (73444/73444), 56.36 MiB | 26.89 MiB/s, done.
Resolving deltas: 100% (52210/52210), done.


In [3]:
!pip install ./transformers/

Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 7.3MB/s 
     |████████████████████████████████| 901kB 51.4MB/s 
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2309653 sha256=089772f88ebd5fcab350b688cff53481a8dae6ecfda0df8a8acb642f70e435b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-nns7gj15/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers


In [4]:
pip show transformers

Name: transformers
Version: 4.7.0.dev0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: regex, huggingface-hub, tokenizers, sacremoses, importlib-metadata, filelock, requests, tqdm, numpy, packaging
Required-by: 


In [5]:
!pip install -r ./transformers/examples/pytorch/question-answering/requirements.txt

     |████████████████████████████████| 225kB 7.4MB/s 
     |████████████████████████████████| 245kB 41.6MB/s 
     |████████████████████████████████| 112kB 59.1MB/s 


## download and transform data

In [6]:
!mkdir dataset \
&& cd dataset \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2021-05-25 19:09:34--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  72.7MB/s    in 0.6s    

2021-05-25 19:09:35 (72.7 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [7]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('dataset/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('dataset/dev-v2.0.json')

In [8]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [9]:
from transformers import RobertaTokenizerFast
# tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
tokenizer = RobertaTokenizerFast.from_pretrained('deepset/roberta-base-squad2')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [10]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

## to PyTorch dataset

In [11]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

## load model

#### for loading not trained model, only it's structure

In [12]:
# from transformers import RobertaConfig, RobertaForQuestionAnswering

# # Initializing a RoBERTa configuration
# config = RobertaConfig()
# config = RobertaConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_attention_heads=12,
#     num_hidden_layers=6,
#     type_vocab_size=1,
# )

# # Initializing a model from the configuration
# model = RobertaForQuestionAnswering(config=config)

#### for loading the pretrained model

In [ ]:
# from transformers import RobertaForQuestionAnswering
# model = RobertaForQuestionAnswering.from_pretrained("roberta-base")

#### mount Google drive

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### load previously saved model from Google drive to continue training

In [62]:
path = "./drive/MyDrive/roberta/roberta.pth"
# model = torch.load(path, map_location=torch.device('cpu')) # kai baigias GPU :(
model = torch.load(path)
model.eval()

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

## training

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [16]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

from tqdm.auto import tqdm
num_training_steps = len(train_loader)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    progress_bar = tqdm(range(num_training_steps))
    i = 0
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        i += 1
        progress_bar.update(1)
        if i%200 == 0:
          path = "./drive/MyDrive/roberta/roberta.pth"
          torch.save(model, path)
          print('Model saved - {}/54.'.format(i/200))

model.eval()

Model saved - 1.0/54.
Model saved - 2.0/54.
Model saved - 3.0/54.
Model saved - 4.0/54.
Model saved - 5.0/54.
Model saved - 6.0/54.
Model saved - 7.0/54.
Model saved - 8.0/54.
Model saved - 9.0/54.
Model saved - 10.0/54.
Model saved - 11.0/54.
Model saved - 12.0/54.
Model saved - 13.0/54.
Model saved - 14.0/54.
Model saved - 15.0/54.
Model saved - 16.0/54.
Model saved - 17.0/54.
Model saved - 18.0/54.
Model saved - 19.0/54.
Model saved - 20.0/54.
Model saved - 21.0/54.
Model saved - 22.0/54.
Model saved - 23.0/54.
Model saved - 24.0/54.
Model saved - 25.0/54.
Model saved - 26.0/54.
Model saved - 27.0/54.
Model saved - 28.0/54.
Model saved - 29.0/54.
Model saved - 30.0/54.
Model saved - 31.0/54.
Model saved - 32.0/54.
Model saved - 33.0/54.
Model saved - 34.0/54.
Model saved - 35.0/54.
Model saved - 36.0/54.
Model saved - 37.0/54.
Model saved - 38.0/54.
Model saved - 39.0/54.
Model saved - 40.0/54.
Model saved - 41.0/54.
Model saved - 42.0/54.
Model saved - 43.0/54.
Model saved - 44.0/5

KeyboardInterrupt: ignored

## QA part

In [63]:
model.to('cpu')
model.eval()

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

In [64]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# tokenizer_m = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
# model_m = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in Transformers?",
    "What does Transformers provide?",
    "Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs, return_dict=False)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: How many pretrained models are available in Transformers?
Answer: 

Question: What does Transformers provide?
Answer: 🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding

Question: Transformers provides interoperability between which frameworks?
Answer: Transformers provides interoperability between which frameworks?</s></s>
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…)



In [65]:
from transformers import pipeline

# Open and read the article
question = "What is the capital of the Netherlands?"
context = r"The four largest cities in the Netherlands are Amsterdam, Rotterdam, The Hague and Utrecht.[17] Amsterdam is the country's most populous city and nominal capital,[18] while The Hague holds the seat of the States General, Cabinet and Supreme Court.[19] The Port of Rotterdam is the busiest seaport in Europe, and the busiest in any country outside East Asia and Southeast Asia, behind only China and Singapore."

# Generating an answer to the question in context
# qa = pipeline("question-answering")
qa = pipeline('question-answering', model=model, tokenizer=tokenizer)
answer = qa(question=question, context=context)

# Print the answer
print(f"Question: {question}")
print(f"Answer: '{answer['answer']}' with score {answer['score']}")

Question: What is the capital of the Netherlands?
Answer: 'Netherlands are Amsterdam, Rotterdam, The Hague and Utrecht' with score 0.019375959411263466
